In [1]:
pip install bitstring

In [2]:
from itertools import cycle

import bitstring
import numpy as np
from glob import glob
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.cluster import estimate_bandwidth, MeanShift


# Get trainticket paths
paths = glob('data/eshopper/*.csv')

# Shuffle the paths
np.random.seed(42)
np.random.shuffle(paths)

# Create a cycle iterator
iterator = cycle(paths)

# Initialize the result dictionary
results = {'train_set': [], 'test_set': [], 'f1': []}


for train_path in iterator:
    # Get test path
    test_path = next(iterator)

    # Get training set name
    train_set = train_path.split('/')[-1].split('.')[0]

    # Get test set name
    test_set = test_path.split('/')[-1].split('.')[0]

    # Read train and test data
    df_train = pd.read_csv(train_path)
    df_test = pd.read_csv(test_path)

    # Extract features
    X_train = df_train.iloc[:, :-1]
    
    # Convert labels to binary (0: the request is not affected by performance issues, 1: the request is affected by a performance issue)
    y_train = [1 if label in [1, 2] else 0 for label in df_train.iloc[:, -1]]
    
    # Extract features
    X_test = df_test.iloc[:, :-1]
    # Convert labels to binary (0: the request is not affected by performance issues, 1: the request is affected by a performance issue)
    y_test = [1 if label in [1, 2] else 0 for label in df_test.iloc[:, -1]]

    # Train a Decision Tree classifier
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_test)

    # Calculate F1 score (average='weighted' to deal with class imbalance)
    f1 = f1_score(y_test, y_pred)
    
    # Append the result to the dictionary
    results['train_set'].append(train_set)
    results['test_set'].append(test_set)
    results['f1'].append(f1)

    # Stop if all the train sets have been used
    if len(results['f1']) == len(paths):
        break


# Create a dataframe from the results
df_results = pd.DataFrame(results)



# Search space contruction

In [3]:
#df_results
X_train[X_train['Latency']>450]


#We gonna save the bandwith and the split points determined by the meanshift algo
bandwith_dict = {}
split_point_dict = {}
conversion_RPC_to_index = {}
index = 0
# For each RPC qui calculate potential split points
for column in X_train.columns:

    #We don't consider latency as a feature
    if column != 'Latency':
        index+=1
        conversion_RPC_to_index[index] = column
        data_rpc_column = X_train[column].values.reshape(-1,1)

        bandwith_dict[index] = estimate_bandwidth(data_rpc_column)

        clustering = MeanShift(bandwidth=bandwith_dict[index]).fit(data_rpc_column)


        # We create a range of points for each rpc to then detect the frontier between the clusters
        x_min = np.min(data_rpc_column)
        x_max = np.max(data_rpc_column)
        range_rpc = np.linspace(x_min,x_max)

        n = len(range_rpc)

        predict = clustering.predict(range_rpc.reshape(-1,1))

        split_points = [x_min]

        #for each point of the range we only save the frontier values
        for i in range(n-1):
            if predict[i] != predict[i+1]:
                split_points.append(range_rpc[i+1])

        split_points.append(x_max)

        split_point_dict[index] = split_points


        # TODO  reject clusters without enough points ie -cluster- <|R_pos| * 0.05






# Precomputation

In [4]:
import random

def extract_R_pos_R_neg(Resquests,Lslo):

    """
    We extract the two subset of request regarding their latency
    
    """
    Rpos = X_train[X_train['Latency']>Lslo]
    Rneg = X_train[X_train['Latency']<=Lslo]



    return Rpos, Rneg

    
def precomputation(split_point_dict,Rpos,Rneg,conversion_RPC_to_index):


    """
    For each pair <j,t> with j the RPC index and t one of the split point of the RPC we generate a list of 
    Bool to check if each request is above or under the split point of the considered RPC
    """
    B_pos = {}
    B_neg = {}

    

    for j in split_point_dict.keys():
            for t in split_point_dict[j]:


                B_pos[(j,t)] = []

                for index,request in Rpos.iterrows():
                        
                    B_pos[(j,t)].append(request[conversion_RPC_to_index[j]]>t)
                B_neg[(j,t)] =  []
                for index,request in Rneg.iterrows():
                    B_neg[(j,t)].append(request[conversion_RPC_to_index[j]]>t)

    return B_pos,B_neg


def boolList2BinString(liste):
    """ 
    Just a translator from boolean list to bitstring
    """


    return '0b' + ''.join(['1' if x else '0' for x in liste])

def H_pos_H_neg(B_pos,B_neg):

    """ 
    Final step of the precomputation, we translate each list of Bool to the bit string we'll use later to compute precision and recall
    """
    H_pos, H_neg = {}, {}
     
    for key, value in B_pos.items():
        H_pos[key] = bitstring.BitArray(boolList2BinString(value))
    for key, value in B_neg.items():
        H_neg[key] = bitstring.BitArray(boolList2BinString(value))
     
    return H_pos, H_neg

In [5]:
Rpos, Rneg = extract_R_pos_R_neg(X_train,450)

H_pos, H_neg = H_pos_H_neg(*precomputation(split_point_dict,Rpos,Rneg,conversion_RPC_to_index))

requests = X_train

In [6]:
def bts_evaluate_predicate(predicate, H_pos,H_neg):

    """ 
    For one predicate <j,emin,emax> we get the bitstring corresponding to <j,emin> and <j,emax> in the positive and negative set and apply the
    bitwise operation proposed in the article
    """


    j,emin,emax = predicate

    B_pos_min, B_pos_max = H_pos[(j,emin)],H_pos[(j,emax)]
    B_neg_min, B_neg_max = H_neg[(j,emin)],H_neg[(j,emax)]

    B_pos = B_pos_min & (~ B_pos_max)
    B_neg = B_neg_min & (~ B_neg_max)
    
    return B_pos, B_neg

def bts_evaluate_pattern(pattern, H_pos,H_neg):
    """
    For each predicate in the pattern we get the evaluation bitstring and then compute the one 
    corresponding to the pattern
    """
    B_pos_list = []
    B_neg_list = []

    for predicate in pattern:
        B_pos, B_neg = bts_evaluate_predicate(predicate, H_pos,H_neg)
        B_pos_list.append(B_pos)
        B_neg_list.append(B_neg)

    res_B_pos =  B_pos_list[0]
    res_B_neg =  B_neg_list[0]

    for i in range(1,len(B_pos_list)):
        res_B_pos = res_B_pos & B_pos_list[i]
        res_B_neg = res_B_neg & B_neg_list[i]

    return res_B_pos, res_B_neg

def bts_evaluate_pattern_set(pattern_set, H_pos,H_neg):
    """
    For each pattern in the pattern set we get the evaluation bitstring and then compute the one 
    corresponding to the pattern set
    """
    B_pos_list = []
    B_neg_list = []

    for pattern in pattern_set:
        B_pos, B_neg = bts_evaluate_pattern(pattern, H_pos,H_neg)
        B_pos_list.append(B_pos)
        B_neg_list.append(B_neg)

    res_B_pos =  B_pos_list[0]
    res_B_neg =  B_neg_list[0]

    for i in range(1,len(B_pos_list)):
        res_B_pos = res_B_pos | B_pos_list[i]
        res_B_neg = res_B_neg | B_neg_list[i]

    return res_B_pos, res_B_neg

def compute_tp_fp(pattern_set,H_pos,H_neg):

    """ 
    We count the number of ones in each bitstring and extrapole precision and recall
    """
    
    B_pos, B_neg = bts_evaluate_pattern_set(pattern_set, H_pos,H_neg)
    tp,fp,fn = B_pos.bin.count('1') , B_neg.bin.count('1'), B_neg.bin.count('0')

    #print(tp,fp,fn,B_pos, B_neg )
    if tp == 0:
        precision = recall = 0
    else:
        precision = tp / ( tp + fp )
        recall = tp / (tp + fn)


    return precision, recall

def Fbeta_score(pattern_set,beta,H_pos,H_neg):

    """
    From one pattern set, we compute Fbeta socre from precision and recall
    """

    precision, recall = compute_tp_fp(pattern_set,H_pos,H_neg)
    if precision == recall == 0:
        score = 0
    else :
        score = (1+beta**2)*(precision*recall)/((beta**2 * precision)+recall)


    return score

def compute_latency_dissimilarity(pattern_set, H_pos,H_neg,requests):

    """
    for each pattern of the set we stock the latencies of the positive and negative dominated requests and then compute the sum(sum(Lat-mean))


    """


    diss_set = 0
    for pattern in pattern_set:
        latencies = []
        B_pos, B_neg = bts_evaluate_pattern(pattern, H_pos,H_neg)

        for i in range(len(B_pos.bin)):
            if B_pos[i] : latencies.append(requests.iloc[i]['Latency'])

        for i in range(len(B_neg.bin)):
            if B_neg[i] : latencies.append(requests.iloc[i]['Latency'])

        if latencies :
            mean = sum(latencies)/len(latencies)
        else : 
            mean = 0

        diss = sum([(lat-mean)**2 for lat in latencies])
        diss_set += diss

    return diss_set


def fitness(pattern_set, H_pos, H_neg, requests):


    """
    We return the tuple (precision,recall,latency_dissimilarity)
    """

    precision, recall = compute_tp_fp(pattern_set,H_pos,H_neg)
    latency_dissimilarity = compute_latency_dissimilarity(pattern_set, H_pos,H_neg,requests)

    return (precision, recall, latency_dissimilarity)



In [7]:
a = bitstring.BitArray('0b0100')

print(type(a.bin))

<class 'str'>


# Genectic algorith

In [8]:
from random import sample


class ParetoFront:


    def __init__(self,pop_size,split_point_dict) -> None:
        self.best_pop = [generate_individual(1,1,split_point_dict) for _ in range(pop_size)]
        self.best_fit = [0 for _ in range(pop_size)]


    def update(self,P,P_fitness):

        
        for individual, fitness in zip(P,P_fitness):


            i = np.argmin(self.best_fit)
            if fitness > self.best_fit[i]:
                self.best_fit[i] = fitness
                self.best_pop[i] = individual


            

    


def genetic_algorith(pop_size,indiv_size,pattern_size,gmax,p,q):

    PF = ParetoFront(pop_size=pop_size, split_point_dict=split_point_dict)
    P = initialise_population(pop_size,indiv_size,pattern_size,split_point_dict)
    P_fitness = Fbeta(P)
    PF.update(P, P_fitness)

    # We make gmax generation 
    for g in range(gmax):
        
        Q = []
        n = len(P)
        # For each individual [pattern_set] in the population
        for _ in range(n):
            r = random.random()
            

            # crossover
            if r < p:
            
                s1,s2 = sample(P,k=2)
                sprime = crossover(s1,s2)

            # mutation
            elif r < q+p:
                s = sample(P,k=1)[0]
                sprime = mutation(s,q,pattern_size,split_point_dict)

            # selection/reproduction
            else:
                
                sprime = sample(P,k=1)[0]

            
            Q.append(sprime)
        
        Pprime = P+Q
        rank_list, diss = rank(Pprime)
        print(rank_list)
        # We sort the total popultion alogn their rank and if ranks are equals the latency dissimilarity is taken into account
        Pprime_indexed = [[i,Pprime[i]] for i in range(len(Pprime))]
        sorted(Pprime_indexed, key = lambda x : (rank_list[x[0]], diss[x[0]]))

        P = [element[1] for element in Pprime_indexed]
        
        P = P[0:n]


        
        P_fitness = Fbeta(P)
        PF.update(P, P_fitness)
        print("La taille de la population à la genration {0} : {1}".format(g,len(P)))

        #print("La population à la generation {0} : {1}".format(g,P))
        print("La fitness à la generation {0} : {1}".format(g,P_fitness))


def rank(population):

    """ 
    Input:
        - population : list of pattern set

    Output:
        - rank list : rank of each pattern set
        - latency dissmilarity : lat_diss of each pattern set 
    """
    n = len(population)
    fitness_list = np.array([(0.0,0.0,0.0) for _ in range(n)])
    eq_dict = {}

    rank = 0
    rank_list = [0 for _ in range(n)]

    pop_copy = population.copy()

    for i in range(n):
        fitness_list[i] = fitness(population[i], H_pos, H_neg, requests)
        eq_dict[str(population[i])] = i

    while pop_copy:
        rank += 1
        non_dominated_pattern = []
        for pattern_set in pop_copy:
            prec, rec, lat = fitness_list[eq_dict[str(pattern_set)]]
            dominated = False
            for pattern_dom in pop_copy:
                prec_dom, rec_dom, lat_dom = fitness_list[eq_dict[str(pattern_dom)]]
                if prec_dom > prec and rec_dom > rec and lat_dom > lat :
                    dominated = True
                    break
            if not dominated:
                non_dominated_pattern.append(pattern_set)
                rank_list[eq_dict[str(pattern_set)]] = rank
        
        for element in non_dominated_pattern:
            pop_copy.remove(element)


    return rank_list, list(fitness_list[:,2])
    

def crossover(s1,s2):

    s1copy = s1.copy()
    s2copy = s2.copy()
    res = []
    while s1copy and s2copy:
        res.append(s1copy[0])
        res.append(s2copy[0])
        s1copy.pop(0)
        s2copy.pop(0)
    if s1copy:
        res += s1copy
    elif s2copy:
        res += s2copy

    cut = int(random.uniform(2,len(res)))
    return res[0:cut]


def mutation(s,q,pattern_size,split_point_dict):
    sprime = s.copy()
    r = random.uniform(0,3)

    if r<1: #remove one pattern
        pattern = random.sample(sprime,k=1)[0]
        sprime.remove(pattern)
    elif r<2: #add one pattern
        pattern = generate_pattern(pattern_size,split_point_dict)
        sprime.append(pattern)
    else: #split one pattern
        pattern = random.sample(sprime,k=1)[0]
        p1,p2 = split(pattern,split_point_dict)
        sprime.remove(pattern)
        sprime.append(p1)
        sprime.append(p2)

    for i in range(len(sprime)):
        r = random.uniform(0,1)
        if r<q:
            if r<q/2:#remove one predicate
                
                predicate = random.sample(sprime[i],k=1)[0]
                sprime[i].remove(predicate)

            else: #add mutation
                predicate = generate_predicate(split_point_dict,len(split_point_dict.keys()))
                sprime[i].append(predicate)
        


    return sprime



def split(pattern,split_point_dict):

    predicate = generate_predicate(split_point_dict,len(split_point_dict.keys()))
    j,e_min,e_max = predicate

    Ej = split_point_dict[j]
    if predicate in pattern:
        e_min_prime = e_min
        e_max_prime = e_max
        p_prime = pattern.copy()
        p_prime.remove(predicate)
    else:
        e_min_prime = Ej[0]
        e_max_prime = Ej[-1]
        p_prime = pattern.copy()

    Ej = [ej for ej in Ej if ej < e_max_prime]
    reduced_Ej = [ej for ej in Ej if ej>= e_min_prime]
    t = sample(reduced_Ej,k = 1)[0]

    p_1 = (j,e_min_prime,t)
    p_2 = (j,t,e_max_prime)
    P1 = p_prime + [p_1]
    P2 = p_prime + [p_2]


    return P1,P2


def initialise_population(pop_size,indiv_size,pattern_size,split_point_dict):



    pop = [generate_individual(indiv_size,pattern_size,split_point_dict) for _ in range(pop_size)]
    
    
    return pop



def generate_individual(indiv_size,pattern_size,split_point_dict):
    

    
    indiv = [generate_pattern(pattern_size,split_point_dict) for k in range(indiv_size)]

    return indiv

def generate_pattern(pattern_size,split_point_dict):
    
    pattern = [(0.0,0.0,0.0) for _ in range(pattern_size)]
    nbr_rpc = len(split_point_dict.keys())

    for i in range(pattern_size):
        pat = generate_predicate(split_point_dict,nbr_rpc)

        pattern[i] = pat

    
    return list(pattern)


def generate_predicate(split_point_dict,nbr_rpc):

    
    j = int(np.random.uniform(1,nbr_rpc))
    
    try:
        index_min = random.choice(range(len(split_point_dict[j])-1))
    except Exception as e:
        index_min = 0
    try : 
        index_max = random.choice(range(index_min+1,len(split_point_dict[j])))
    except Exception as e:
        index_max = 1

    try :
        e_min, e_max = split_point_dict[j][index_min],split_point_dict[j][index_max]
    except Exception as e:
        print(e,index_min,index_max,split_point_dict[j])



    return (j,e_min,e_max)

def Fbeta(pop):
    

    Fbeta_pop = [Fbeta_score(ind,0.1,H_pos,H_neg) for ind in pop]

    return  Fbeta_pop




In [9]:
print(split_point_dict)

{1: [2, 12.612244897959185, 34.89795918367347, 54], 2: [1, 20.0, 43.0, 50], 3: [2, 3.0816326530612246, 4.163265306122449, 5.244897959183674, 6.326530612244898, 24.714285714285715, 43.102040816326536, 44.183673469387756, 45.26530612244898, 46.34693877551021, 47.42857142857143, 48.51020408163266, 52.83673469387755, 55], 4: [41, 75.12244897959184, 117], 5: [1, 10.73469387755102, 22.63265306122449, 38.85714285714286, 54], 6: [1, 16.510204081632654, 33.6530612244898, 41], 7: [103, 181.0, 247.0, 337.0, 397]}


In [10]:
genetic_algorith(pop_size = 20, indiv_size=15, pattern_size=15,gmax = 5,p = 0.3,q = 0.2)



[1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
La taille de la population à la genration 0 : 20
La fitness à la generation 0 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
La taille de la population à la genration 1 : 20
La fitness à la generation 1 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]
La taille de la population à la genration 2 : 20
La fitness à la generation 2 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
La taille de la population à la genration 3 : 20
La fitness à la generation 3 : [0, 0, 0, 0, 